In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session ## 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from  xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
import time

In [ ]:
train_df = pd.read_csv('../input/bike-sharing-demand/train.csv')
test_df = pd.read_csv('../input/bike-sharing-demand/test.csv')


In [ ]:
test_df

In [ ]:
train_df

In [ ]:
print(train_df.isnull().values.any() ,'\n',test_df.isnull().values.any())

In [ ]:
print(train_df.info() ,'\n',test_df.info())


In [ ]:
test = pd.DataFrame(test_df)
test

In [ ]:
train = pd.DataFrame(train_df)
train

# EDA

**Train data**

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'], format = '%Y-%m-%dT%H:%M:%S')
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour

#Get days names
# train_df['weekday']=pd.DatetimeIndex(train_df['datetime']).day_name()
train

In [ ]:
train = train_df.drop('datetime' , axis=1)
train

In [ ]:

train[["season", "count"]].groupby(['season'], as_index=False).sum().sort_values(by='count', ascending=False)

In [ ]:
train[["holiday", "count"]].groupby(['holiday'], as_index=False).sum().sort_values(by='count', ascending=False)

In [ ]:
train[["workingday", "count"]].groupby(['workingday'], as_index=False).sum().sort_values(by='count', ascending=False)

**Test data**

In [ ]:
test['datetime'] = pd.to_datetime(test['datetime'], format = '%Y-%m-%dT%H:%M:%S')
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour

test

In [ ]:
plt.subplots(figsize=(15,15))
sns.heatmap(train.corr(), mask=np.zeros_like(train.corr(), dtype=bool),
            square=True, annot=True)
plt.show()

**Drop the columns that most correlated to the 'count' column to avoid redundancy**

In [ ]:
train = pd.DataFrame(train)
train.drop(['casual' , 'registered' ],axis=1,inplace=True)


In [ ]:
plt.bar(train['hour'], train['count'])

plt.xlabel('hour')
plt.ylabel('Number of bikes')
plt.show()


From the previous plot we can see the bikes rental demands ratio is higher at day hours more than the night hours

In [ ]:
c=[]
for i in test['hour']:
    
    if i>= 6 or i<= 1 :
        c.append("Day")
    else:
        c.append("Night")
test['DayorNight']=c
test['DayorNight']=pd.factorize(test['DayorNight'])[0].reshape(-1, 1)
c=[]


test = test.drop('datetime' , axis =1)
test

In [ ]:
c=[]
for i in train['hour']:
    
    if i>= 6 or i<= 1 :
        c.append("Day")
    else:
        c.append("Night")
train['DayorNight']=c
train['DayorNight']=pd.factorize(train['DayorNight'])[0].reshape(-1, 1)
c=[]
train.head(2)

In [ ]:
x = train
x =x.drop('count',axis=1)
x

In [ ]:
test

In [ ]:
# y= train['count']

y=np.log1p(train['count'])
y = pd.DataFrame(y)
y.var()

sns.histplot(y);
plt.title("Bike Count");


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x , y, train_size=0.8, test_size=0.2, random_state = 0)

# **Compare models using LazyPredict Library**

In [ ]:
# ! pip install lazypredict


In [ ]:
# from lazypredict.Supervised import LazyRegressor
# reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
# models , predictions = reg.fit(x_train, x_val, y_train, y_val)

# print(models) 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error


**HistGradientBoostingRegressor**

In [ ]:
# from sklearn.experimental import enable_hist_gradient_boosting  
# from sklearn.ensemble import HistGradientBoostingRegressor


# est = HistGradientBoostingRegressor()

# est.fit(x_train, y_train)

# score_train_est = est.score(x_train, np.ravel(y_train,order='C'))  
# print("Training score: ", score_train_est)

# scores_est = cross_val_score(est, x_train, y_train,cv=10)
# print("Mean cross-validation score: %.2f" % scores_est.mean())


# y_pred_est = est.predict(x_val)
# print('Validation score' , est.score(x_val , y_val))

# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_squared_log_error


# rmsle_est = mean_squared_log_error(y_pred_est ,y_val )
# print("RMSLE: %.2f" % rmsle_est)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
# print(xgbr)

xgbr.fit(x_train, y_train)

score_xgbr = xgbr.score(x_train, y_train)  
print("Training score: ", score_xgbr)

scores_xgbr = cross_val_score(xgbr, x_train, y_train,cv=10)
print("Mean cross-validation score: %.2f" % scores_xgbr.mean())

 
y_pred_xgbr = xgbr.predict(x_val)
print('Validation score' , xgbr.score(x_val , y_val))

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error


rmsle_xgbr = mean_squared_log_error(y_pred_xgbr ,y_val )
print("RMSLE: %.2f" % rmsle_xgbr)


In [ ]:
# from sklearn.ensemble import ExtraTreesRegressor

# et = ExtraTreesRegressor() 
# # print(et)

# et.fit(x_train, y_train)

# score_train_et = et.score(x_train, y_train)  
# print("Training score: ", score_train_et)

# scores_et = cross_val_score(et, x_train, y_train,cv=10)
# print("Mean cross-validation score: %.2f" % scores_et.mean())


# y_pred_et = et.predict(x_val)
# print('Validation score' , et.score(x_val , y_val))

# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_squared_log_error


# rmsle_et = mean_squared_log_error(y_pred_et ,y_val )
# print("RMSLE: %.2f" % rmsle_et)

In [ ]:
# y_pred_est_test = xgbr.predict(test_final) #to be saved

pred =np.round(np.expm1(xgbr.predict(test))).astype(int)
pred = pd.DataFrame({"datetime": test_df["datetime"],"count": np.fix(pred)})
pred.shape

In [ ]:
pred

In [ ]:
# y_pred_est_test = y_pred_est_test.drop('datetime' , axis=1)
pred.to_csv('submission.csv', index=False)  #